> Martin Fridrich 03/2022

# Initial exploratory data analysis - Olist dataset



### Goals

We strive to examine the sales part of the dataset in such a way, so we can design and implement a meaningful processing pipeline and use the results in downstream models. To meet the goal, we analyze the size + packaging (num of records, cols, dtypes, memory requirements, coverage amongst the tables) and explore a few business aspects (basic stats, outliers, trends, customer & supplier & item, demand curve).

Unfortunately, we find that the data at hand are not suitable for customer churn modeling. Only 19 customers with 5 or more transactions. See the print-outs.

### Olist data model
<img src="imgs/olist-data-model.png" width="800px" />


# Data loading

In [1]:
DATA_DIR = "../data/raw/olist/"

import pandas as pd
import numpy as np

## Orders

In [2]:
orders = pd.read_csv(DATA_DIR+"olist_orders_dataset.csv")
print("\nPeek at basic properties.")
display(orders.head(3)); display(orders.info());


Peek at basic properties.


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


None

In [3]:
# clean the dtypes
# to dt
dt_cols = ["order_purchase_timestamp", "order_approved_at", "order_delivered_carrier_date",
    "order_delivered_customer_date","order_estimated_delivery_date"]
for c in dt_cols:
    orders[c] = pd.to_datetime(orders[c])
# to cat
orders["order_status"] = orders["order_status"].astype("category")

print("\nPeek at .info() again.")
display(orders.info())
print("\nPeek at status freqs.")
display(orders.order_status.value_counts());
# dt range
print("\nOrders ranging from {} to {}.".format(orders.order_purchase_timestamp.min(),
    orders.order_purchase_timestamp.max()))


Peek at .info() again.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  category      
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](5), object(2)
memory usage: 5.4+ MB


None


Peek at status freqs.


delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: order_status, dtype: int64


Orders ranging from 2016-09-04 21:15:19 to 2018-10-17 17:30:18.


In [9]:
#BIG FUCK RIGHT HERE
orders.groupby("customer_id").order_id.count().sort_values().tail(5)

customer_id
558de2f6863bf7588338479d614522a3    1
558db91121d704e92ef88de10aa722fd    1
558db5ecbdc2c47fa8b3f8ae3b8a0453    1
559f9d302aa523b2dd718e956f248db5    1
ffffe8b65bbe3087b653a978c870db99    1
Name: order_id, dtype: int64

In [14]:
# STILL NO LUCK, OBVIOUSLY WE CANNOT USE THIS DATA
customers = pd.read_csv(DATA_DIR+"olist_customers_dataset.csv")
orders.merge(customers, on="customer_id", how="left").groupby("customer_unique_id").\
    order_id.count().sort_values(ascending=False).head(20)

customer_unique_id
8d50f5eadf50201ccdcedfb9e2ac8455    17
3e43e6105506432c953e165fb2acf44c     9
6469f99c1f9dfae7733b25662e7f1782     7
ca77025e7201e3b30c44b472ff346268     7
1b6c7548a2a1f9037c1fd3ddfed95f33     7
12f5d6e1cbf93dafd9dcc19095df0b3d     6
de34b16117594161a6a89c50b289d35a     6
63cfc61cee11cbe306bff5857d00bfe4     6
f0e310a6839dce9de1638e0fe5ab282a     6
47c1a3033b8b77b3ab6e109eb4d5fdf3     6
dc813062e0fc23409cd255f7f53c7074     6
4e65032f1f574189fb793bac5a867bbc     5
b4e4f24de1e8725b74e4a1f4975116ed     5
394ac4de8f3acb14253c177f0e15bc58     5
5e8f38a9a1c023f3db718edcf926a2db     5
35ecdf6858edc6427223b64804cf028e     5
74cb1ad7e6d5674325c1f99b5ea30d82     5
fe81bb32c243a86b2f86fbf053fe6140     5
56c8638e7c058b98aae6d74d2dd6ea23     5
795c1622cf7a53d63d324e862349d01c     4
Name: order_id, dtype: int64